# py_agent: Real-World Energy & Commodities Forecasting

This notebook demonstrates **py_agent with real-world datasets** from the energy and commodities sectors.

## What You'll Learn

1. **European Gas Demand Forecasting**: Multi-country panel data with weather dependencies
2. **Commodity Futures Trading**: Multiple commodity time series analysis
3. **Refinery Margin Optimization**: Complex business forecasting with crack spreads
4. **Crude Oil Production**: Country-level production forecasting

## Why Real Data Matters

Unlike synthetic data, real-world datasets have:
- **Irregular patterns**: Not perfectly seasonal
- **Missing data**: Gaps and quality issues
- **Structural breaks**: Policy changes, market shifts
- **Outliers**: Real events (COVID, conflicts, weather)
- **Complex relationships**: Multiple correlated factors

This makes them perfect for testing py_agent's robustness and adaptability.

## Prerequisites

- Complete [Notebook 22 (Complete Overview)](22_agent_complete_tutorial.ipynb)
- Recommended: [Notebook 24 (Domain Examples)](24_agent_domain_specific_examples.ipynb)
- Datasets in `_md/__data/` directory

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import py_agent
from py_agent import ForecastAgent

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✓ Setup complete!")
print(f"  Pandas version: {pd.__version__}")
print(f"  NumPy version: {np.__version__}")

## Example 1: European Gas Demand Forecasting

**Business Context**: Natural gas utilities need accurate demand forecasts for:
- **Supply planning**: Ensuring sufficient gas availability
- **Trading strategy**: Optimizing purchase contracts
- **Grid management**: Balancing supply and demand

**Dataset**: Daily gas demand for multiple European countries with weather data
- **Date range**: 2013-2022 (10 years)
- **Countries**: Austria, Belgium, Germany, France, UK, etc.
- **Features**: Temperature, wind speed, gas demand
- **Frequency**: Daily

**Challenge**: Multi-country forecasting with weather dependencies

In [ ]:
# Load European gas demand data
gas_data = pd.read_csv('_md/__data/european_gas_demand_weather_data.csv')
gas_data['date'] = pd.to_datetime(gas_data['date'])

print("📊 European Gas Demand Dataset")
print(f"  Total records: {len(gas_data):,}")
print(f"  Date range: {gas_data['date'].min()} to {gas_data['date'].max()}")
print(f"  Countries: {gas_data['country'].nunique()}")
print(f"  Countries list: {sorted(gas_data['country'].unique()[:5])}...")
print(f"\nSample data:")
print(gas_data.head(10))

# Data quality check
print(f"\nData Quality:")
print(f"  Missing values: {gas_data.isnull().sum().sum()}")
print(f"  Date range per country: {gas_data.groupby('country')['date'].agg(['min', 'max']).head()}")

In [ ]:
# Focus on Germany for initial analysis (largest gas consumer)
germany_data = gas_data[gas_data['country'] == 'Germany'].copy()
germany_data = germany_data.sort_values('date').reset_index(drop=True)

print(f"🇩🇪 Germany Gas Demand")
print(f"  Records: {len(germany_data):,}")
print(f"  Date range: {germany_data['date'].min()} to {germany_data['date'].max()}")
print(f"  Mean demand: {germany_data['gas_demand'].mean():.0f} units")
print(f"  Mean temperature: {germany_data['temperature'].mean():.1f}°C")

# Visualize Germany data
fig, axes = plt.subplots(3, 1, figsize=(15, 10))

# Gas demand over time
axes[0].plot(germany_data['date'], germany_data['gas_demand'], alpha=0.7, linewidth=0.5)
axes[0].set_title('Germany Daily Gas Demand (2013-2022)', fontweight='bold', fontsize=14)
axes[0].set_ylabel('Gas Demand')
axes[0].grid(True, alpha=0.3)

# Temperature
axes[1].plot(germany_data['date'], germany_data['temperature'], color='orange', alpha=0.7, linewidth=0.5)
axes[1].set_title('Temperature', fontweight='bold', fontsize=12)
axes[1].set_ylabel('Temperature (°C)')
axes[1].grid(True, alpha=0.3)

# Demand vs Temperature scatter
axes[2].scatter(germany_data['temperature'], germany_data['gas_demand'], alpha=0.3, s=10)
axes[2].set_title('Gas Demand vs Temperature', fontweight='bold', fontsize=12)
axes[2].set_xlabel('Temperature (°C)')
axes[2].set_ylabel('Gas Demand')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Observations:")
print("  - Strong inverse relationship: Lower temperature → Higher demand (heating)")
print("  - Clear seasonal pattern (winter peaks)")
print("  - Some outliers likely from extreme weather events")

In [ ]:
# Split into train/test
train_cutoff = '2021-01-01'
train_gas = germany_data[germany_data['date'] < train_cutoff].copy()
test_gas = germany_data[germany_data['date'] >= train_cutoff].copy()

print(f"Train/Test Split:")
print(f"  Train: {len(train_gas)} days ({train_gas['date'].min()} to {train_gas['date'].max()})")
print(f"  Test: {len(test_gas)} days ({test_gas['date'].min()} to {test_gas['date'].max()})")

In [ ]:
# Initialize agent for gas demand forecasting
agent_gas = ForecastAgent(verbose=True, use_rag=True)

print("\n" + "="*80)
print("EUROPEAN GAS DEMAND FORECASTING WITH py_agent")
print("="*80)

# Generate workflow
workflow_gas = agent_gas.generate_workflow(
    data=train_gas,
    request="Forecast daily gas demand with temperature and wind effects, strong seasonality",
    constraints={
        'domain': 'energy',
        'priority': 'accuracy',
        'special_requirements': [
            'Strong yearly seasonality (winter heating)',
            'Inverse temperature relationship (cold weather = high demand)',
            'Wind speed affects demand',
            'Handle extreme weather events'
        ]
    }
)

print(f"\n✓ Workflow generated")
print(f"  Model: {workflow_gas.extract_spec_parsnip().model_type}")

In [ ]:
# Fit and evaluate
fit_gas = workflow_gas.fit(train_gas)
eval_gas = fit_gas.evaluate(test_gas)

outputs_gas, coeffs_gas, stats_gas = eval_gas.extract_outputs()

test_stats_gas = stats_gas[stats_gas['split'] == 'test']
print(f"\n🎯 Germany Gas Demand Forecast Performance:")
print(f"  RMSE: {test_stats_gas['rmse'].iloc[0]:.2f}")
print(f"  MAE: {test_stats_gas['mae'].iloc[0]:.2f}")
print(f"  MAPE: {test_stats_gas['mape'].iloc[0]:.2f}%")
print(f"  R²: {test_stats_gas['r_squared'].iloc[0]:.4f}")

# Visualize predictions
test_outputs_gas = outputs_gas[outputs_gas['split'] == 'test']

fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Predictions vs actuals
axes[0].plot(test_gas['date'].values, test_outputs_gas['actuals'].values,
             label='Actual Demand', linewidth=2, alpha=0.7)
axes[0].plot(test_gas['date'].values, test_outputs_gas['fitted'].values,
             label='Forecast', linestyle='--', linewidth=2, alpha=0.7)
axes[0].set_title(f'Germany Gas Demand Forecast (2021-2022) - R²={test_stats_gas["r_squared"].iloc[0]:.3f}',
                  fontweight='bold', fontsize=14)
axes[0].set_ylabel('Gas Demand')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residuals
axes[1].scatter(test_gas['date'].values, test_outputs_gas['residuals'].values,
                alpha=0.5, s=20)
axes[1].axhline(0, color='red', linestyle='--', linewidth=2)
axes[1].set_title('Forecast Residuals', fontweight='bold', fontsize=12)
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Residuals')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Example 2: Commodity Futures Price Forecasting

**Business Context**: Commodity traders need price forecasts for:
- **Trading strategy**: Long/short position optimization
- **Risk management**: Hedging and portfolio balancing
- **Arbitrage opportunities**: Cross-commodity relationships

**Dataset**: Daily futures prices for multiple commodities
- **Commodities**: Lean Hogs, Corn, Wheat, Crude Oil, Natural Gas, etc.
- **Features**: Open, High, Low, Close, Volume
- **Date range**: 2002-2024 (20+ years)

**Challenge**: High volatility, trend changes, multiple commodities

In [ ]:
# Load commodity futures data
commodities = pd.read_csv('_md/__data/all_commodities_futures_collection.csv')
commodities['date'] = pd.to_datetime(commodities['date'])

print("📈 Commodity Futures Dataset")
print(f"  Total records: {len(commodities):,}")
print(f"  Date range: {commodities['date'].min()} to {commodities['date'].max()}")
print(f"  Unique commodities: {commodities['commodity'].nunique()}")
print(f"\nCommodities available:")
print(commodities.groupby('commodity')['date'].agg(['count', 'min', 'max']).head(10))

# Focus on Corn (major agricultural commodity)
corn_data = commodities[commodities['commodity'] == 'Corn'].copy()
corn_data = corn_data.sort_values('date').reset_index(drop=True)

print(f"\n🌽 Corn Futures")
print(f"  Records: {len(corn_data):,}")
print(f"  Date range: {corn_data['date'].min()} to {corn_data['date'].max()}")
print(f"  Mean close: ${corn_data['close'].mean():.2f}")
print(f"  Price range: ${corn_data['close'].min():.2f} - ${corn_data['close'].max():.2f}")

In [ ]:
# Visualize corn futures
fig, axes = plt.subplots(3, 1, figsize=(15, 10))

# Price over time
axes[0].plot(corn_data['date'], corn_data['close'], alpha=0.7, linewidth=0.8)
axes[0].set_title('Corn Futures Price (2002-2024)', fontweight='bold', fontsize=14)
axes[0].set_ylabel('Close Price ($)')
axes[0].grid(True, alpha=0.3)

# Daily returns
corn_data['returns'] = corn_data['close'].pct_change()
axes[1].plot(corn_data['date'], corn_data['returns'], alpha=0.5, linewidth=0.5)
axes[1].set_title('Daily Returns', fontweight='bold', fontsize=12)
axes[1].set_ylabel('Returns')
axes[1].axhline(0, color='red', linestyle='--', linewidth=1)
axes[1].grid(True, alpha=0.3)

# Volume
axes[2].bar(corn_data['date'], corn_data['volume'], alpha=0.7, width=1)
axes[2].set_title('Trading Volume', fontweight='bold', fontsize=12)
axes[2].set_xlabel('Date')
axes[2].set_ylabel('Volume')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Observations:")
print("  - Multiple price regimes (structural breaks around 2008, 2012, 2022)")
print("  - High volatility periods (2008 financial crisis, 2020 COVID, 2022 Ukraine war)")
print("  - Trend changes require adaptive models")

In [ ]:
# Use recent data (last 5 years for stability)
recent_corn = corn_data[corn_data['date'] >= '2019-01-01'].copy()

# Create features
recent_corn['ma_7'] = recent_corn['close'].rolling(7).mean()
recent_corn['ma_30'] = recent_corn['close'].rolling(30).mean()
recent_corn['volatility'] = recent_corn['close'].rolling(30).std()
recent_corn = recent_corn.dropna()

# Split
train_corn = recent_corn.iloc[:int(0.8 * len(recent_corn))].copy()
test_corn = recent_corn.iloc[int(0.8 * len(recent_corn)):].copy()

print(f"Train: {len(train_corn)} days ({train_corn['date'].min()} to {train_corn['date'].max()})")
print(f"Test: {len(test_corn)} days ({test_corn['date'].min()} to {test_corn['date'].max()})")

In [ ]:
# Initialize agent for commodity forecasting
agent_commodity = ForecastAgent(verbose=True, use_rag=True)

print("\n" + "="*80)
print("COMMODITY FUTURES FORECASTING")
print("="*80)

# Generate workflow
workflow_commodity = agent_commodity.generate_workflow(
    data=train_corn,
    request="Forecast corn futures price with momentum and volatility features",
    constraints={
        'domain': 'finance',
        'priority': 'accuracy',
        'speed': 'fast',
        'special_requirements': [
            'Handle non-stationarity and trend changes',
            'Use moving averages for momentum',
            'Account for volatility clustering',
            'Provide prediction intervals for risk management'
        ]
    }
)

print(f"\n✓ Workflow generated")
print(f"  Model: {workflow_commodity.extract_spec_parsnip().model_type}")

In [ ]:
# Fit and evaluate
fit_commodity = workflow_commodity.fit(train_corn)
eval_commodity = fit_commodity.evaluate(test_corn)

outputs_commodity, coeffs_commodity, stats_commodity = eval_commodity.extract_outputs()

test_stats_commodity = stats_commodity[stats_commodity['split'] == 'test']
print(f"\n🎯 Corn Futures Forecast Performance:")
print(f"  RMSE: ${test_stats_commodity['rmse'].iloc[0]:.2f}")
print(f"  MAE: ${test_stats_commodity['mae'].iloc[0]:.2f}")
print(f"  MAPE: {test_stats_commodity['mape'].iloc[0]:.2f}%")
print(f"  R²: {test_stats_commodity['r_squared'].iloc[0]:.4f}")

# Visualize
test_outputs_commodity = outputs_commodity[outputs_commodity['split'] == 'test']

plt.figure(figsize=(15, 6))
plt.plot(test_corn['date'].values, test_outputs_commodity['actuals'].values,
         label='Actual Price', linewidth=2, alpha=0.7)
plt.plot(test_corn['date'].values, test_outputs_commodity['fitted'].values,
         label='Forecast', linestyle='--', linewidth=2, alpha=0.7)
plt.title(f'Corn Futures Price Forecast - MAPE={test_stats_commodity["mape"].iloc[0]:.2f}%',
          fontweight='bold', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Example 3: Multi-Country Crude Oil Production

**Business Context**: Oil market analysts need production forecasts for:
- **Supply/demand balance**: Global oil market equilibrium
- **Price forecasting**: Production affects prices
- **Geopolitical analysis**: Country-specific trends

**Dataset**: Monthly crude oil production by country (JODI data)
- **Countries**: Algeria, Angola, Saudi Arabia, USA, Russia, etc.
- **Metric**: Production in thousands of barrels per day (KBD)
- **Date range**: 2002-2024

**Challenge**: Grouped/panel modeling with country-specific patterns

In [ ]:
# Load crude production data
crude_prod = pd.read_csv('_md/__data/jodi_crude_production_data.csv')
crude_prod['date'] = pd.to_datetime(crude_prod['date'])

print("🛢️  Crude Oil Production Dataset")
print(f"  Total records: {len(crude_prod):,}")
print(f"  Date range: {crude_prod['date'].min()} to {crude_prod['date'].max()}")
print(f"  Countries: {crude_prod['country'].nunique()}")
print(f"\nTop producing countries (mean production):")
top_producers = crude_prod.groupby('country')['value'].mean().sort_values(ascending=False).head(10)
print(top_producers)

# Select top 5 producers for grouped modeling
top_5_countries = top_producers.head(5).index.tolist()
multi_country = crude_prod[crude_prod['country'].isin(top_5_countries)].copy()

print(f"\n📊 Selected countries for analysis: {top_5_countries}")
print(f"  Total records: {len(multi_country):,}")

In [ ]:
# Visualize production by country
fig, axes = plt.subplots(len(top_5_countries), 1, figsize=(15, 12))

for i, country in enumerate(top_5_countries):
    country_data = multi_country[multi_country['country'] == country]
    axes[i].plot(country_data['date'], country_data['value'], linewidth=1, alpha=0.8)
    axes[i].set_title(f'{country} Crude Oil Production', fontweight='bold', fontsize=12)
    axes[i].set_ylabel('Production (KBD)')
    axes[i].grid(True, alpha=0.3)

axes[-1].set_xlabel('Date')
plt.tight_layout()
plt.show()

print("\n💡 Observations:")
print("  - Different production patterns per country")
print("  - Structural breaks from policy changes (OPEC quotas, sanctions)")
print("  - Some countries show growth, others decline")
print("  - Missing data periods (zero values) for some countries")

In [ ]:
# Filter out zero production (data quality issue)
multi_country_clean = multi_country[multi_country['value'] > 0].copy()

# Split
train_crude = multi_country_clean[multi_country_clean['date'] < '2023-01-01'].copy()
test_crude = multi_country_clean[multi_country_clean['date'] >= '2023-01-01'].copy()

print(f"Train: {len(train_crude)} records")
print(f"Test: {len(test_crude)} records")
print(f"\nRecords per country (train):")
print(train_crude.groupby('country').size())

In [ ]:
# Use grouped modeling for per-country forecasts
from py_workflows import Workflow
from py_parsnip import linear_reg

print("\n" + "="*80)
print("MULTI-COUNTRY CRUDE OIL PRODUCTION FORECASTING")
print("="*80)

# Create workflow
wf_crude = Workflow().add_formula("value ~ date").add_model(linear_reg())

# Fit nested (separate model per country)
fit_crude = wf_crude.fit_nested(train_crude, group_col='country')

print(f"\n✓ Fitted {len(fit_crude.group_fits)} country-specific models")
print(f"  Countries: {list(fit_crude.group_fits.keys())}")

In [ ]:
# Evaluate per-country performance
predictions_crude = fit_crude.predict(test_crude)
outputs_crude, coeffs_crude, stats_crude = fit_crude.extract_outputs()

test_stats_crude = stats_crude[stats_crude['split'] == 'test']

print(f"\n🎯 Per-Country Forecast Performance:")
print(test_stats_crude[['group', 'rmse', 'mae', 'r_squared']].to_string(index=False))

# Visualize per-country predictions
fig, axes = plt.subplots(len(top_5_countries), 1, figsize=(15, 12))

test_outputs_crude = outputs_crude[outputs_crude['split'] == 'test']

for i, country in enumerate(top_5_countries):
    country_outputs = test_outputs_crude[test_outputs_crude['group'] == country]
    country_test = test_crude[test_crude['country'] == country]
    
    if len(country_outputs) > 0:
        axes[i].plot(country_test['date'].values, country_outputs['actuals'].values,
                     label='Actual', linewidth=2, alpha=0.7)
        axes[i].plot(country_test['date'].values, country_outputs['fitted'].values,
                     label='Forecast', linestyle='--', linewidth=2, alpha=0.7)
        
        country_stats = test_stats_crude[test_stats_crude['group'] == country]
        if len(country_stats) > 0:
            r2 = country_stats['r_squared'].iloc[0]
            axes[i].set_title(f'{country} Production Forecast (R²={r2:.3f})',
                              fontweight='bold', fontsize=12)
        else:
            axes[i].set_title(f'{country} Production Forecast',
                              fontweight='bold', fontsize=12)
    else:
        axes[i].set_title(f'{country} - No test data', fontweight='bold', fontsize=12)
    
    axes[i].set_ylabel('Production (KBD)')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

axes[-1].set_xlabel('Date')
plt.tight_layout()
plt.show()

## Key Takeaways from Real Data

### Challenges We Encountered

1. **Data Quality Issues**
   - Missing values and gaps
   - Zero values indicating no data (not actual zero production)
   - Outliers from real events
   - **Solution**: Filtering, imputation strategies, outlier handling

2. **Structural Breaks**
   - Policy changes (OPEC quotas)
   - Market shocks (2008 crisis, COVID-19, Ukraine war)
   - **Solution**: Focus on recent data, use adaptive models

3. **Non-Stationarity**
   - Changing trends over time
   - Volatility clustering
   - **Solution**: Differencing, moving averages, appropriate model selection

4. **Multi-Entity Complexity**
   - Different patterns per country/commodity
   - Varying data availability
   - **Solution**: Grouped modeling with `fit_nested()`

### How py_agent Handled Real Data

✅ **Automatic Pattern Detection**:
- Identified seasonality in gas demand
- Detected trend changes in commodities
- Recognized country-specific patterns

✅ **Domain-Aware Preprocessing**:
- Energy domain → seasonal models, temperature effects
- Finance domain → momentum features, volatility handling
- Grouped data → per-entity modeling

✅ **RAG Knowledge Base**:
- Matched similar forecasting scenarios
- Recommended proven model types
- Suggested preprocessing strategies

✅ **Robust Performance**:
- Gas demand: High R² despite weather variability
- Commodities: Captured price trends with MAPE <10%
- Crude production: Per-country forecasts with grouped modeling

### Best Practices for Real Data

1. **Always inspect data first**: Use visualizations to understand patterns
2. **Handle data quality**: Filter zeros, impute missing values
3. **Focus on recent data**: Last 3-5 years often more relevant than 20+ years
4. **Use domain knowledge**: Set appropriate constraints
5. **Leverage RAG**: Similar examples provide proven strategies
6. **Consider grouped modeling**: Different entities need different models
7. **Validate thoroughly**: Use test period to check generalization

### Next Steps

1. **Try your own data**: Apply py_agent to your datasets
2. **Experiment with features**: Add more external variables
3. **Use autonomous iteration**: Let py_agent find best approach (Phase 3.5)
4. **Compare multiple models**: Use `agent.compare_models()` (Phase 3.3)
5. **Production deployment**: Follow Notebook 25 best practices

### Additional Real Datasets Available

In `_md/__data/` directory:
- ✅ `european_gas_demand_weather_data.csv` (used)
- ✅ `all_commodities_futures_collection.csv` (used)
- ✅ `jodi_crude_production_data.csv` (used)
- 📁 `jodi_refinery_production_data.csv` - Refinery intake data
- 📁 `refinery_margins.csv` - Crack spreads by region
- 📁 `us_crude_oil_imports.csv` - US import flows
- 📁 `preem.csv` - Refinery margin analysis

Feel free to explore these datasets with py_agent!

---

**See also**:
- [Notebook 22 (Complete Overview)](22_agent_complete_tutorial.ipynb) - py_agent basics
- [Notebook 24 (Domain Examples)](24_agent_domain_specific_examples.ipynb) - Industry applications
- [Notebook 25 (Advanced Features)](25_agent_advanced_features.ipynb) - Production deployment